In [6]:
import psycopg2
import pandas as pd
import re
from datetime import date

In [ ]:
def get_cursor():
    conn = psycopg2.connect(
    database="legislative_testimony",
    user="brianrennie",
    password="")
    cursor = conn.cursor()
    return cursor

cursor = get_cursor()

In [7]:
bill_df = pd.read_csv('../data/all-history-actions.csv', index_col=0)

bill_df

,action,branch,date
id,,,
H1,Referred to the committee on House Ways and Means,House,2021-02-25T16:46:19.933Z
H1,"Reported, in part, by H4000",House,2021-04-15T04:00:00.000Z
H10,Referred to the committee on Public Service,House,2021-04-13T15:59:37.223Z
H10,Senate concurred,Senate,2021-04-13T15:59:37.223Z
H10,Hearing scheduled for 07/28/2021 from 01:00 PM...,Joint,2021-07-22T14:53:10.290Z
...,...,...,...
s2729,Recommended new text for H4126,Senate,2022-02-28T05:00:00.000Z
s2729,Rules suspended,Senate,2022-02-28T05:00:00.000Z
s2729,Substituted as a new text for H4126,Senate,2022-02-28T05:00:00.000Z


In [19]:
bill_df.describe()

,action,branch,date
count,45472,45472,45472
unique,5511,4,13698
top,Senate concurred,House,2021-03-30T03:59:59.000Z
freq,6484,19519,4988


In [20]:
bill_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45472 entries, H1 to s3049
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   action  45472 non-null  object
 1   branch  45472 non-null  object
 2   date    45472 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [9]:
action_counts = bill_df['action'].value_counts().to_dict()

action_counts

enacted_count = bill_df[(bill_df['action'].str.count('(^Enacted)')>0)]

enacted_count

reading = bill_df[(bill_df['action'].str.count('(^Read)')>0)]

reading

,action,branch,date
id,,,
H100,Read second and ordered to a third reading,House,2021-04-15T15:16:10.033Z
H100,Read third and passed to be engrossed,House,2021-04-28T04:00:00.000Z
H100,"Read, rules suspended, read second and ordered...",Senate,2021-04-29T04:00:00.000Z
H100,Read third (title changed) and passed to be en...,Senate,2021-06-03T15:12:00.000Z
H1065,Read second and ordered to a third reading,House,2022-03-24T15:19:09.420Z
...,...,...,...
S9,Read third,Senate,2021-01-28T05:00:00.000Z
S9,Read second and ordered to a third reading,House,2021-01-28T05:00:00.000Z
S9,Read third,House,2021-01-28T05:00:00.000Z


In [49]:
action_counts = bill_df['action'].value_counts().to_dict()

branch_action_count = bill_df['branch'].value_counts()

branch_action_count




House        19519
Senate       17888
Joint         7845
Executive      220
Name: branch, dtype: int64

In [24]:
def number_enacted(year):
    cursor = get_cursor()
    cursor.execute('''with enacted_Actions as (SELECT EXTRACT(YEAR FROM date) as bill_year,
                        COUNT(CASE WHEN action lIKE '%Enacted%' then 'yes' ELSE NULL
                         end) as enacted                   
                      from bill_history 
                      GROUP BY 1 )
                      Select * from enacted_Actions WHERE bill_year = ?
                      ''', (year.bill_year))
    result = cursor.fetchall()
    return result

number_enacted(2021)

AttributeError: 'int' object has no attribute 'bill_year'